# 🔄 Inferência Recorrente de Vendas - CVC Lojas

## 🎯 Objetivo
Execução periódica (Semanal/Mensal) para gerar novas previsões de vendas.
Este notebook não treina modelos. Ele carrega o modelo produtivo (`All-in-One`) e gera forecast baseando-se no histórico mais recente.

## ⚙️ Fluxo de Execução
1.  **Context Loading:** Carrega os últimos 90 dias de vendas (Janela de Contexto) do Data Lake.
2.  **Model Loading:** Baixa o modelo do Unity Catalog (`Usage: Production`).
3.  **Inference:** O Wrapper `UnifiedForecaster` recebe o contexto, normaliza, prevê e desnormaliza.
4.  **Persistence:** Salva os resultados na tabela `bip_vprevisao_lojas_futuro`.


In [0]:
# --- SETUP INICIAL ---
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.getcwd())

from src.validation.config import Config
from src.validation.data import DataIngestion
from datetime import timedelta, date
import mlflow
from mlflow.tracking import MlflowClient
import pyspark.sql.functions as F
import pandas as pd
from datetime import datetime
import numpy as np
import pandas as pd
import pyspark.sql.functions as F

# Darts classes para Wrapper funcionar
from darts import TimeSeries

# Configs Spark
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")
client = MlflowClient()

In [0]:
# --- 1. DEFINIÇÃO DA JANELA DE CONTEXTO ---
# Simulando "Hoje" (em prod usar date.today())
#today = date.today()
today = datetime.strptime("2025-02-01", "%Y-%m-%d").date() 

# Janela de Lookback: precisamos de histórico suficiente para os Lags do modelo (ex: 60-90 dias)
context_days = 90 
start_context = today - timedelta(days=context_days)

# Config Dinâmica
config = Config(spark)
config.DATA_START = start_context.strftime("%Y-%m-%d")
config.INGESTION_END = today.strftime("%Y-%m-%d")
config.SCHEMA = "cvc_pred"

print(f"📅 Data de Referência (Hoje): {today}")
print(f"🔎 Carregando contexto a partir de: {config.DATA_START}")

In [0]:
# --- 2. CARREGAMENTO DO CONTEXTO (SPARK) ---
# Reutilizamos a classe DataIngestion para garantir consistência nas features
ingestion = DataIngestion(spark, config)

print("   ⏳ Lendo dados históricos recentes...")
df_context_spark = ingestion.create_training_set()

# Filtro de Segurança e Seleção de Colunas
df_context_spark = df_context_spark.filter(
    F.col("DATA").between(config.DATA_START, config.INGESTION_END)
)

# Traz para Pandas (Driver) - Volume pequeno pois é só janela recente
df_context_pd = df_context_spark.toPandas()
df_context_pd['data'] = pd.to_datetime(df_context_pd['data'])

# Injeta parametro 'n' para o wrapper saber o horizonte desejado
FORECAST_HORIZON = 35
df_context_pd['n'] = FORECAST_HORIZON

In [0]:
# --- 3. CARREGAMENTO DO MODELO ---
model_name = f"{config.CATALOG}.{config.SCHEMA}.cvc_lojas_forecast_production"
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}@Champion")
print("✅ Modelo Carregado!")
mv = client.get_model_version_by_alias(name=model_name, alias="Champion")

print("Modelo:", model_name)
print("Versão do modelo:", mv.version)
print("Run ID:", mv.run_id)
print("Current stage:", mv.current_stage)
print("Description:", mv.description)

In [0]:
# --- PREPARAÇÃO DE DADOS DE MERCADO ---

# 1. Carrega a tabela de suporte (a mesma usada no treino)
df_market_spark = spark.table(f"{config.CATALOG}.{config.SCHEMA}.historico_suporte_loja")

# 2. Pivota para criar as colunas (IPCA, DOLAR...)
# Importante: O nome das colunas deve bater exatamente com o treino
df_market_wide = (df_market_spark
    .groupBy("data")
    .pivot("metricas")
    .agg(F.sum("valor"))
    .na.fill(0.0))

# 3. Converte para Pandas para fazer o merge local (já que a inferência é pandas)
pdf_market = df_market_wide.toPandas()
pdf_market['data'] = pd.to_datetime(pdf_market['data']).dt.strftime('%Y-%m-%d')

In [0]:
# --- MONTAGEM FINAL DO CONTEXTO (MERCADO + FUTURO) ---

# ==============================================================================
# 1. PREPARAÇÃO DE DADOS DE MERCADO (Incorporado)
# ==============================================================================
print("📊 Carregando dados de mercado (IPCA, Dólar, Feriados)...")

# Carrega a tabela de suporte
df_market_spark = spark.table(f"{config.CATALOG}.{config.SCHEMA}.historico_suporte_loja")

# Pivota para formato Wide (Colunas: IPCA, DOLAR...)
df_market_wide = (df_market_spark
    .groupBy("data")
    .pivot("metricas")
    .agg(F.sum("valor"))
    .na.fill(0.0)
)

# Traz para Pandas
pdf_market = df_market_wide.toPandas()
# Garante string YYYY-MM-DD para join seguro
pdf_market['data'] = pd.to_datetime(pdf_market['data']).dt.strftime('%Y-%m-%d')

# ==============================================================================
# 2. CRIAÇÃO DO ESQUELETO FUTURO E MERGE
# ==============================================================================
print("⏳ Montando esqueleto de datas futuras para previsão...")

# Garante datetime no contexto original
df_context_pd['data'] = pd.to_datetime(df_context_pd['data'])
last_date = df_context_pd['data'].max()

# Gera datas futuras (+15 dias de buffer para segurança dos lags)
future_horizon_days = FORECAST_HORIZON + 15
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_horizon_days, freq='D')

# Identifica colunas estáticas para replicar (UF, Cluster, etc.)
static_cols = ['codigo_loja', 'cluster_loja', 'sigla_uf', 'tipo_loja', 'modelo_loja']
static_cols = [c for c in static_cols if c in df_context_pd.columns]

# Pega a última "foto" de cada loja
df_stores_reference = df_context_pd.sort_values('data').groupby('codigo_loja')[static_cols].tail(1)

# Cross Join: Todas as Lojas x Todas as Datas Futuras
df_future_skeleton = df_stores_reference.assign(key=1).merge(
    pd.DataFrame({'data': future_dates, 'key': 1}), 
    on='key'
).drop('key', axis=1)

# Sinaliza que é futuro (Target NaN) e define horizonte
df_future_skeleton['target_vendas'] = np.nan
df_future_skeleton['n'] = FORECAST_HORIZON

# Une Histórico + Futuro
df_full_timeline = pd.concat([df_context_pd, df_future_skeleton], ignore_index=True)
df_full_timeline['data'] = df_full_timeline['data'].dt.strftime('%Y-%m-%d')

# MERGE FINAL: Aplica os dados de mercado nas datas futuras
df_inference_final = pd.merge(
    df_full_timeline, 
    pdf_market, 
    on='data', 
    how='left'
)

# Preenche buracos eventuais no mercado com 0.0
cols_mercado = [c for c in pdf_market.columns if c != 'data']
df_inference_final[cols_mercado] = df_inference_final[cols_mercado].fillna(0.0)
# Ajuste de tipos
df_inference_final['codigo_loja'] = df_inference_final['codigo_loja'].astype(str)

In [0]:
datetime.now()

In [0]:
# --- PREVISÃO ---
print("🔮 Gerando previsões...")
df_inference_final = df_inference_final.loc[:, ~df_inference_final.columns.duplicated()]
forecast_df = loaded_model.predict(df_inference_final)
forecast_df['version_model'] = mv.version
forecast_df['description_model'] = mv.description
forecast_df['model_name'] = model_name
forecast_df['data_reference'] = datetime.now()

In [0]:
# --- 5. PERSISTÊNCIA (WRITE BACK) ---
output_table = f"{config.CATALOG}.{config.SCHEMA}.previsao_lojas_futuro"

print(f"💾 Salvando resultados em: {output_table}")
(spark.createDataFrame(forecast_df)
 .write
 .format("delta")
 .mode("append") # Append histórico de previsões
 .option("mergeSchema", "true")
 .saveAsTable(output_table)
)
spark.sql(f"OPTIMIZE {output_table}")
print("✨ Sucesso! Dados salvos e otimizados.")